In [ ]:
from bs4 import BeautifulSoup
import requests
import json


In [ ]:
def resize(im):
    v = im.split("/")
    return "/".join(v[0:4]+["350", "350"]+v[6:])

In [ ]:
def getData(q,size):
    db=[]
    try:
        for page in range(1,size+1):
            content = requests.get(f"https://www.flipkart.com/search?q={q}&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_6_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_1_6_na_na_na&as-pos=1&as-type=RECENT&requestId=795b7b75-1b4c-4d54-87a2-7dba22c60b66&as-backfill=on&page="+str(page)).content
            soup = BeautifulSoup(content, "html")
            for i in soup.find_all("a", class_="_1fQZEK"):
                p = {}
                m = requests.get("https://www.flipkart.com"+i["href"]).content
                ms = BeautifulSoup(m, "html")
                nm = ms.find("span", class_="B_NuCI").text.split(" ")
                p["name"] = " ".join(nm)
                p["brand"] = nm[0]
                p["quantity"] = "5"
                p["id"] = p["brand"].lower()+str(len(db)+1)
                p["price"] = ms.find("div", class_="_30jeq3 _16Jk6d").text.replace("\u20b9","")
                mrp = ms.find("div", class_="_3I9_wc _2p6lqe")
                if mrp != None:
                    p["mrp"] = mrp.text
                else:
                    p["mrp"] = p["price"]
                p["images"] = [resize(str(i["src"]))
                               for i in ms.find_all("img", class_="q6DClP")]
                p["specifications"] = [i.text for i in ms.find_all("li", class_="_21Ahn-")]
                db.append(p)
    except Exception:
        print(p)

    return db


In [ ]:
def run(name,pages,save):
    data=getData(name,pages)
    if save:
        with open(name+"_products.json", "w") as file:
            file.write(json.dumps(data, indent=5))
        print(f"Saved {len(data)} products")
        return
    return data


In [ ]:
run("apple",2,False)